In [1]:
import os

from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import timm

import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from src.model import NW_CNN
from src.dataset import CropsPytorchDataset
from src.utils import Averager

In [2]:
le = LabelEncoder()
le.fit(['human-hold-bicycle', 'human-ride-bicycle', 'human-ride-motorcycle', 'human-walk-bicycle', 'human-walk-motorcycle', 'human-hold-motorcycle'])
train_dataset = CropsPytorchDataset(img_dir='data/annotated_data/clips/', anno_file='data_anno/training_data.csv', label_encoder=le)
val_dataset = CropsPytorchDataset(img_dir='data/annotated_data/clips/', anno_file='data_anno/testing_data.csv', label_encoder=le)

train_dataloader = DataLoader(train_dataset, batch_size=40, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=40, shuffle=True)

In [3]:
out_dir = 'products/models'
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [4]:
model = timm.create_model('resnet18', num_classes = 6, in_chans=1)
# model = NW_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

device = torch.device('cuda')

In [5]:
#In case you want to continue training from checkpoint
model = torch.load('products/models/NW_CNN_checkpoint.pth')
model.train()
model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [6]:
train_losses = []
val_losses = []
train_loss_hist = Averager()
val_loss_hist = Averager()
best_valid_loss=float('inf')
max_epochs = 40

for epoch in range(20, max_epochs):  # loop over the dataset multiple times
    print(f"\nEPOCH {epoch} of {max_epochs}")
    train_loss_hist.reset()
    prog_bar = tqdm(train_dataloader, total=len(train_dataloader), leave=True)
    for inputs, labels in prog_bar:
        labels = torch.squeeze(labels, dim=1).long().to(device)
        inputs = inputs.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss_hist.send(loss)
        loss.backward()
        optimizer.step()

   
    print(f"Epoch #{epoch} train loss: {train_loss_hist.value:.3f}")
    train_losses.append(train_loss_hist.value)

    val_loss_hist.reset()  
    
    prog_bar = tqdm(val_dataloader, total=len(val_dataloader), leave=True)
    for inputs, labels in prog_bar:
        labels = torch.squeeze(labels, dim=1).long().to(device)
        inputs = inputs.to(device)
        outputs = model(inputs)
        with torch.no_grad():
            loss = criterion(outputs, labels)
        val_loss_hist.send(loss)      
     
    print(f"Epoch #{epoch} validation loss: {val_loss_hist.value:.3f}")
    val_losses.append(val_loss_hist.value)

    #Saving best model
    if val_loss_hist.value < best_valid_loss:
        best_valid_loss = val_loss_hist.value
        print(f"\nBest validation loss: {best_valid_loss}")
        print(f"\nSaving best model for epoch: {epoch}\n")
        torch.save(model, os.path.join('products/models', f'NW_CNN_checkpoint.pth'))

    
print('Finished Training')


EPOCH 20 of 40


100%|██████████| 192/192 [00:18<00:00, 10.67it/s]


Epoch #20 train loss: 0.612


100%|██████████| 88/88 [00:07<00:00, 11.65it/s]


Epoch #20 validation loss: 0.555

Best validation loss: 0.5548872351646423

Saving best model for epoch: 20


EPOCH 21 of 40


...

Epoch #38 validation loss: 0.554

EPOCH 39 of 40


100%|██████████| 192/192 [00:17<00:00, 10.84it/s]


Epoch #39 train loss: 0.611


100%|██████████| 88/88 [00:07<00:00, 11.56it/s]

Epoch #39 validation loss: 0.550
Finished Training


In [9]:
model = torch.load('products/models/NW_CNN_checkpoint.pth')
model.to(torch.device('cpu'))
model.eval();

In [10]:
from torchmetrics import Accuracy, Recall, Precision, ConfusionMatrix, MetricCollection
metrics = MetricCollection([Accuracy(task="multiclass", num_classes=6, average='macro'),
                            Precision(task="multiclass", num_classes=6, average='macro'),
                            Recall(task="multiclass", num_classes=6, average='macro'),
                            ConfusionMatrix(task="multiclass", num_classes=6)])

for i, (crop, target) in enumerate(val_dataloader):
    prediction = torch.argmax(model(crop), dim=1)
    target = torch.squeeze(target)
    metrics(prediction, target)

print(metrics.compute())
metrics.reset()


{'MulticlassAccuracy': tensor(0.2615), 'MulticlassPrecision': tensor(0.3536), 'MulticlassRecall': tensor(0.2615), 'MulticlassConfusionMatrix': tensor([[  34,    0,  122,    0,   27,    0],
        [   0,    0,    2,    0,    0,    0],
        [  14,    0, 2930,    0,   15,    0],
        [   1,    0,  195,    0,    0,    0],
        [  10,    0,  122,    0,   20,    0],
        [   0,    0,    0,    0,    0,    0]])}
